In [ ]:
import cv2

# 1) Open a connection to the default camera (index 0).
#    If you have multiple cameras, you may need to use index 1, 2, etc.
camera = cv2.VideoCapture(0)

# 2) Read a frame from the camera.
ret, frame = camera.read()

if ret:
# 3) If reading was successful, save the frame to a file.
    save_path = r"F:\Thesis\my_captured_image.jpg"  # raw string for Windows path
    cv2.imwrite(save_path, frame)
    print(f"Image saved to {save_path}")
else:
    print("Failed to capture image from camera.")

# 4) Release the camera resource.
camera.release()

In [ ]:
import socket

def run_server(host='0.0.0.0', port=5000):
    """
    Starts a server socket that listens on the specified host and port.
    On receiving data from a client, it processes the data (example: uppercases it),
    then sends the processed result back.
    """
    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind to the specified host and port
    server_socket.bind((host, port))
    
    # Start listening (allow 1 connection in backlog)
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}...")
    
    # Accept a connection
    conn, addr = server_socket.accept()
    print(f"Connected by {addr}")

    with conn:
        # Receive data from the client
        data = conn.recv(1024)
        if not data:
            print("No data received. Closing connection.")
        else:
            # Decode the data from bytes to string
            received_str = data.decode('utf-8')
            print(f"Received from client: {received_str}")

            # ----------------------------------------------------
            # Your "processing function" can go here.
            # For demonstration, we'll just uppercase the data.
            # Replace it with something meaningful for your scenario.
            # ----------------------------------------------------
            processed_result = received_str.upper()
            
            # Send the processed data back to the client
            conn.sendall(processed_result.encode('utf-8'))
            print(f"Sent back to client: {processed_result}")
    
    # Close the server socket
    server_socket.close()

run_server(host='192.168.2.19', port=5000)

In [ ]:
import socket
from opentrons.types import Point, Location
import opentrons.execute

def run(protocol):
    # Load labware and pipette
    tiprack = protocol.load_labware('opentron_96_tiprack_1000u', '11')
    pipette = protocol.load_instrument('p1000_single', 'right', tip_racks=[tiprack])
    protocol.home()  # Initial homing

    # Socket server setup
    host = '0.0.0.0'
    port = 5000
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}")

    try:
        while True:
            conn, addr = server_socket.accept()
            print(f"Connected by {addr}")
            with conn:
                while True:
                    data = conn.recv(1024).decode('utf-8').strip()
                    if not data:
                        break
                    print(f"Received command: {data}")

                    # Process command
                    response = "Unknown command"
                    try:
                        if data == 'HOME':
                            protocol.home()
                            response = "Homed"
                        elif data.startswith('MOVE_TO_COORDS'):
                            _, x, y, z = data.split()
                            location = Location(Point(float(x), float(y), float(z)), None)
                            pipette.move_to(location)
                            response = f"Moved to ({x}, {y}, {z})"
                        elif data.startswith('MOVE_TO_WELL'):
                            _, well_name = data.split()
                            well = tiprack.wells_by_name()[well_name]
                            pipette.move_to(well.top())
                            response = f"Moved to {well_name}"
                        elif data == 'EXIT':
                            break
                    except Exception as e:
                        response = f"Error: {str(e)}"

      